In [2]:
#printing the sample data from the dataset
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
file_path = 'C:\\Users\\civin\\Desktop\\DUK\\AIcyber\\Intrusion MLP\\datasetkdd.csv'
df = pd.read_csv(file_path)
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.0,tcp,ftp_data,SF,491.0,0.0,0,0.0,0.0,0.0,...,25.0,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0.0,udp,other,SF,146.0,0.0,0,0.0,0.0,0.0,...,1.0,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,26.0,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0.0,tcp,http,SF,232.0,8153.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0.0,tcp,http,SF,199.0,420.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [ ]:
X = df.drop(columns=['class'])
y = df['class']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# categorical features
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=[object]).columns.tolist()

# label Encoding
for col in categorical_features:
    X[col] = label_encoder.fit_transform(X[col])
# Scale numeric
scaler = MinMaxScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

In [68]:
#Mutual Information
mi_scores = mutual_info_classif(X, y_encoded, random_state=0)
mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
print("\nMutual Information Scores")
print(mi_scores)

#features based on Mutual Information scores
top_fe = mi_scores.index[:15]
print("\nTop features by Mutual Information")
print(top_fe)
X_top_15 = X[top_fe]

#Correlation Matrix 
plt.figure(figsize=(10, 8))
corr_matrix = X_top_15.corr()
plt.show()

# Remove highly correlated features
threshold = 0.9
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]

# Final selected features
final_selected_features = top_fe.drop(to_drop)
print("\nFinal selected features after removing correlated ones:")
print(final_selected_features)
selected_list = final_selected_features.tolist()
print("\nFeatures selected for training:")
print(selected_list)


Mutual Information Scores
src_bytes                      0.542267
dst_bytes                      0.436684
service                        0.432796
flag                           0.338168
same_srv_rate                  0.327304
diff_srv_rate                  0.320296
dst_host_srv_count             0.305503
dst_host_same_srv_rate         0.279291
dst_host_diff_srv_rate         0.259381
count                          0.234714
serror_rate                    0.234034
srv_serror_rate                0.220667
dst_host_srv_diff_host_rate    0.174053
dst_host_same_src_port_rate    0.128850
protocol_type                  0.052840
dtype: float64

Top features by Mutual Information
Index(['src_bytes', 'dst_bytes', 'service', 'flag', 'same_srv_rate',
       'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'count', 'serror_rate', 'srv_serror_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_same_src_port_rate',
       'protocol_type'],
      dtype

<Figure size 1000x800 with 0 Axes>


Final selected features after removing correlated ones:
Index(['src_bytes', 'dst_bytes', 'service', 'flag', 'same_srv_rate',
       'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'count', 'serror_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_same_src_port_rate',
       'protocol_type'],
      dtype='object')

Features selected for training:
['src_bytes', 'dst_bytes', 'service', 'flag', 'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'count', 'serror_rate', 'dst_host_srv_diff_host_rate', 'dst_host_same_src_port_rate', 'protocol_type']


In [69]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# List of top features based on importance from the mutual relation and corelation matrix i choose these features
selected_features = [
    'service', 'dst_host_same_src_port_rate', 'count','serror_rate',
    'protocol_type', 'srv_serror_rate', 'dst_host_srv_diff_host_rate','src_bytes','dst_bytes',  
    'flag', 'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 
    'dst_host_diff_srv_rate']


X = df[selected_features].copy() 

#LabelEncoder 
label_encoder = LabelEncoder()

# Encode categorical features by converrt it into numerical values
X['flag'] = label_encoder.fit_transform(X['flag'])
X['service'] = label_encoder.fit_transform(X['service'])
X['protocol_type'] = label_encoder.fit_transform(X['protocol_type'])

#MinMaxScaler  normalization
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=selected_features)
X_normalized.head()

,service,dst_host_same_src_port_rate,count,serror_rate,protocol_type,srv_serror_rate,dst_host_srv_diff_host_rate,src_bytes,dst_bytes,flag,same_srv_rate,diff_srv_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate
0,0.289855,0.17,0.003914,0.0,0.5,0.0,0.00,3.558064e-07,0.000000e+00,0.9,1.00,0.00,0.098039,0.17,0.03
1,0.637681,0.88,0.025440,0.0,1.0,0.0,0.00,1.057999e-07,0.000000e+00,0.9,0.08,0.15,0.003922,0.00,0.60
2,0.710145,0.00,0.240705,1.0,0.5,1.0,0.00,0.000000e+00,0.000000e+00,0.5,0.05,0.07,0.101961,0.10,0.05
3,0.347826,0.03,0.009785,0.2,0.5,0.2,0.04,1.681203e-07,6.223962e-06,0.9,1.00,0.00,1.000000,1.00,0.00
4,0.347826,0.00,0.058708,0.0,0.5,0.0,0.00,1.442067e-07,3.206260e-07,0.9,1.00,0.00,1.000000,1.00,0.00


In [70]:
#trainig  the model using the MLP classifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


x = df[selected_features]
y = label_encoder.fit_transform(df['class'])

# 70:30 ratio divide
# Split the dataset into training and test
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42)

# MLP model
mlp = MLPClassifier(hidden_layer_sizes=(32, 16), max_iter=300, random_state=42,early_stopping=True, validation_fraction=0.2)
mlp.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(32, 16), max_iter=300,
              random_state=42, validation_fraction=0.2)

In [71]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred = mlp.predict(X_test)

# models performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# evaluation metrics
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9689
Precision: 0.9649
Recall: 0.9756
F1-Score: 0.9702

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.97     21454
           1       0.96      0.98      0.97     23102

    accuracy                           0.97     44556
   macro avg       0.97      0.97      0.97     44556
weighted avg       0.97      0.97      0.97     44556

